<a href="https://colab.research.google.com/github/yeswhos/MSc-Project/blob/master/Control%20System/Decision%20Tree/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
filepath = "drive/My Drive/lenses.txt"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# -*- coding: UTF-8 -*-
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals.six import StringIO
from sklearn import tree
import pandas as pd
import numpy as np
import pydotplus


with open(filepath, 'r') as fr:										#加载文件
	lenses = [inst.strip().split('\t') for inst in fr.readlines()]		#处理文件
lenses_target = []														#提取每组数据的类别，保存在列表里
for each in lenses:
	lenses_target.append(each[-1])
print(lenses_target)



/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


['no lenses', 'soft', 'no lenses', 'hard', 'no lenses', 'soft', 'no lenses', 'hard', 'no lenses', 'soft', 'no lenses', 'hard', 'no lenses', 'soft', 'no lenses', 'no lenses', 'no lenses', 'no lenses', 'no lenses', 'hard', 'no lenses', 'soft', 'no lenses', 'no lenses']


In [3]:
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']			#特征标签		
lenses_list = []														#保存lenses数据的临时列表
lenses_dict = {}														#保存lenses数据的字典，用于生成pandas
for each_label in lensesLabels:											#提取信息，生成字典
	for each in lenses:
		lenses_list.append(each[lensesLabels.index(each_label)])
	lenses_dict[each_label] = lenses_list
	lenses_list = []
#print(lenses_dict)														#打印字典信息
lenses_pd = pd.DataFrame(lenses_dict)									#生成pandas.DataFrame
print(lenses_pd)														#打印pandas.DataFrame


           age prescript astigmatic tearRate
0        young     myope         no  reduced
1        young     myope         no   normal
2        young     myope        yes  reduced
3        young     myope        yes   normal
4        young     hyper         no  reduced
5        young     hyper         no   normal
6        young     hyper        yes  reduced
7        young     hyper        yes   normal
8          pre     myope         no  reduced
9          pre     myope         no   normal
10         pre     myope        yes  reduced
11         pre     myope        yes   normal
12         pre     hyper         no  reduced
13         pre     hyper         no   normal
14         pre     hyper        yes  reduced
15         pre     hyper        yes   normal
16  presbyopic     myope         no  reduced
17  presbyopic     myope         no   normal
18  presbyopic     myope        yes  reduced
19  presbyopic     myope        yes   normal
20  presbyopic     hyper         no  reduced
21  presby

In [4]:
le = LabelEncoder()														#创建LabelEncoder()对象，用于序列化			
for col in lenses_pd.columns:											#序列化
	lenses_pd[col] = le.fit_transform(lenses_pd[col])
print(lenses_pd)														#打印编码信息



    age  prescript  astigmatic  tearRate
0     2          1           0         1
1     2          1           0         0
2     2          1           1         1
3     2          1           1         0
4     2          0           0         1
5     2          0           0         0
6     2          0           1         1
7     2          0           1         0
8     0          1           0         1
9     0          1           0         0
10    0          1           1         1
11    0          1           1         0
12    0          0           0         1
13    0          0           0         0
14    0          0           1         1
15    0          0           1         0
16    1          1           0         1
17    1          1           0         0
18    1          1           1         1
19    1          1           1         0
20    1          0           0         1
21    1          0           0         0
22    1          0           1         1
23    1         

In [5]:
clf = tree.DecisionTreeClassifier(max_depth = 4)						#创建DecisionTreeClassifier()类
clf = clf.fit(lenses_pd.values.tolist(), lenses_target)					#使用数据，构建决策树

dot_data = StringIO()
tree.export_graphviz(clf, out_file = dot_data,							#绘制决策树
					feature_names = lenses_pd.keys(),
					class_names = clf.classes_,
					filled=True, rounded=True,
					special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("drive/My Drive/tree.pdf")												#保存绘制好的决策树，以PDF的形式存储。

										#预测

True

In [6]:
print(clf.predict([[3,0,1,0]]))	

['hard']


In [7]:
import pickle
def storeTree(inputTree, filename):
    '''
    序列化决策树,存入文件
    :param inputTree:
    :param filename:
    :return:
    '''
    
    fw = open(filename,'wb')
    pickle.dump(inputTree,fw)
    fw.close()

In [8]:
treeFile = "/drive/My Drive/tree"
storeTree(clf, r"drive/My Drive/tree.txt")

In [9]:
def grabTree(filename):
    fr = open(filename, 'rb')
    return pickle.load(fr)

In [10]:
mytree = grabTree(r"drive/My Drive/tree.txt")
mytree.predict([[2,0,1,0]])

array(['hard'], dtype='<U9')

In [12]:
print(clf)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
